In [15]:
df = pd.read_csv("test.csv")


df["Max drift mm"] = pd.to_numeric(df["Max drift mm"], errors="coerce")


df = df[df["Max drift mm"].notna()]
df = df[df["Max drift mm"] >= 0].reset_index(drop=True)

print("Clean dataset shape:", df.shape)
print("Minimum drift:", df["Max drift mm"].min())

Clean dataset shape: (308, 23)
Minimum drift: 0.4


In [18]:
import numpy as np
df = pd.read_csv("test.csv")

df = df[
    (df["Max drift mm"].notna()) &   
    (df["Max drift mm"] >= 0) &      
    (np.isfinite(df["Max drift mm"])) 
]

In [1]:
# 1. Imports

import pandas as pd
import numpy as np
import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.inspection import PartialDependenceDisplay

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# 2. Load Data

df = pd.read_csv("train.csv")

target = "Max drift mm"
selected_features = [
    "Period s",
    "Number of floors",
    "PGA g",
    "Magnitude",
    "Distance to fault km",
    "Columns 1-3 I mm4*10^6"
]

X = df[selected_features]
y = df[target]


# Log scaling target
y_log = np.log1p(y)

# 3. Train-Test Split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)

# 4. Bayesian Optimization for XGBoost

def objective_xgb(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 2.0),
        "random_state": 42,
        "tree_method": "hist"
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train, verbose=False)
    preds = model.predict(X_valid)
    return mean_absolute_percentage_error(y_valid, preds)

study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=20)

best_xgb_params = study_xgb.best_params

study_xgb = optuna.create_study(direction="minimize")
study_xgb.optimize(objective_xgb, n_trials=20)

best_xgb_params = study_xgb.best_params
print("Best Params:", best_xgb_params)

model = XGBRegressor(
    **best_xgb_params,
    random_state=42,
    tree_method="hist"
)


model.fit(X_train, y_train)
y_pred = model.predict(X_valid)


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

r2 = r2_score(y_valid, y_pred)
mae = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
mape = mean_absolute_percentage_error(y_valid, y_pred) * 100
accuracy = 100 - mape

print("\n===== Evaluation =====")
print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"Accuracy: {accuracy:.2f}%")


# 5. Define Optimized Models

xgb1 = XGBRegressor(**best_xgb_params)
xgb2 = XGBRegressor(**best_xgb_params)

lgb1 = LGBMRegressor(random_state=42)
lgb2 = LGBMRegressor(random_state=24)

cat = CatBoostRegressor(iterations=1500, depth=8, learning_rate=0.02,
                        verbose=0, random_seed=42)

# 6. Stacking Regressor

stack_model = StackingRegressor(
    estimators=[
        ("xgb1", xgb1),
        ("xgb2", xgb2),
        ("lgb1", lgb1),
        ("lgb2", lgb2),
        ("cat", cat)
    ],
    final_estimator=LinearRegression()
)


# 7. Full Pipeline

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", stack_model)
])


# 8. Train Final Model

pipeline.fit(X_train, y_train)

# 9. Evaluate
preds_log = pipeline.predict(X_valid)
preds = np.expm1(preds_log)
y_true = np.expm1(y_valid)

mape = mean_absolute_percentage_error(y_true, preds)
print(f"Validation MAPE: {mape:.4f}")

# 10. SHAP Explainability

explainer = shap.Explainer(pipeline.predict, X_train)
shap_values = explainer(X_train)
shap.summary_plot(shap_values, X_train)


# 11. Partial Dependence Plots

PartialDependenceDisplay.from_estimator(pipeline, X_train, [0,1,2])
plt.show()

# 12. Safety Classification

def classify_building(drift):
    if drift <= 20:
        return "Safe"
    elif drift <= 50:
        return "Warning"
    else:
        return "Dangerous"


# 13. User Input Prediction

user_input = {}
print("Enter building and seismic properties:")

for feature in selected_features:
    while True:
        try:
            user_input[feature] = float(input(f"{feature}: "))
            break
        except ValueError:
            print("Please enter a numeric value.")

X_user = pd.DataFrame([user_input])

pred_log = pipeline.predict(X_user)[0]
pred = np.expm1(pred_log)
status = classify_building(pred)

print(f"\nPredicted Drift: {pred:.2f} mm -> {status}")

[I 2026-02-22 14:57:16,495] A new study created in memory with name: no-name-6d15f8dc-8cec-4df1-afc7-4e2f47e3cc27
[I 2026-02-22 14:57:18,082] Trial 0 finished with value: 0.06578449584418117 and parameters: {'n_estimators': 597, 'learning_rate': 0.011738748405380507, 'max_depth': 5, 'subsample': 0.7785087173711276, 'colsample_bytree': 0.795226007993106, 'reg_alpha': 0.8911627661581074, 'reg_lambda': 0.5118093621661106}. Best is trial 0 with value: 0.06578449584418117.
[I 2026-02-22 14:57:20,319] Trial 1 finished with value: 0.04808377875191815 and parameters: {'n_estimators': 515, 'learning_rate': 0.021723315135615987, 'max_depth': 8, 'subsample': 0.8370317095759701, 'colsample_bytree': 0.789329938291928, 'reg_alpha': 0.5211910487206577, 'reg_lambda': 0.25282870202357044}. Best is trial 1 with value: 0.04808377875191815.
[I 2026-02-22 14:57:23,674] Trial 2 finished with value: 0.04236251947971315 and parameters: {'n_estimators': 1081, 'learning_rate': 0.024212882605799627, 'max_depth':

Best Params: {'n_estimators': 1218, 'learning_rate': 0.03825610856077764, 'max_depth': 7, 'subsample': 0.7149189498812142, 'colsample_bytree': 0.9371375182730216, 'reg_alpha': 0.15797317169100644, 'reg_lambda': 1.6883427877983932}

===== Evaluation =====
R2 Score: 0.9765
MAE: 0.0953
RMSE: 0.1353
MAPE: 3.69%
Accuracy: 96.31%
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000838 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 322
[LightGBM] [Info] Number of data points in the train set: 2337, number of used features: 6
[LightGBM] [Info] Start training from score 2.717326
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000436 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 322
[LightGBM] [Info] Number of data points in the train set: 2337, number of us

ExactExplainer explainer:  83%|████████▎ | 1936/2337 [22:41<04:42,  1.42it/s]


KeyboardInterrupt: 

In [2]:
# 12. Safety Classification

def classify_building(drift):
    if drift <= 20:
        return "Safe"
    elif drift <= 50:
        return "Warning"
    else:
        return "Dangerous"


# 13. User Input Prediction 
user_input = {}
print("Enter building and seismic properties:")

for feature in selected_features:
    while True:
        try:
            user_input[feature] = float(input(f"{feature}: "))
            break
        except ValueError:
            print("Please enter a numeric value.")

X_user = pd.DataFrame([user_input])

pred_log = pipeline.predict(X_user)[0]
pred = np.expm1(pred_log)
status = classify_building(pred)

print(f"\nPredicted Drift: {pred:.2f} mm -> {status}")

Enter building and seismic properties:


NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.